In [1]:
import requests
import json

TRAPI Documentation: https://github.com/NCATSTranslator/ReasonerAPI

Most TRAPI documents contain a `message` key.  Within that `message` are a `query_graph` denoting the user query,
a `knowledge_graph` consisting of the union of all nodes and edges that match the `query_graph` pattern, and a list of `results` that bind `query_graph` elements to `knowledge_graph` elements.

The following message contains only a `query_graph`.  This query graph consists of 3 nodes connected together in a line.   Two of the nodes (`n00` and `n02`) have specified identifiers, while the middle node of the line does not.  Rather the middle node has a list of `categories` that are acceptable.

This query asks "Find me a Biological Process or Activity, or a Gene, or a Pathway that is related to both `PUBCHEM.COMPOUND:644073` (Buprenorphine) and `HP:0001337` (Tremor).

In [2]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ["PUBCHEM.COMPOUND:644073"],
            "categories": ["biolink:ChemicalEntity"]
          },
          "n01": {
              "categories": ["biolink:BiologicalProcessOrActivity","biolink:Gene","biolink:Pathway"]
          },
          "n02": {
            "ids": ["HP:0001337"],
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


You can also bypass TRAPI entirely and just use cypher to talk to the graph.  There are two instances.  There is one at http://robokopkg.renci.org which has a cypher browser on it, or you can write cypher and post it there.

In [26]:
from neo4j import GraphDatabase

In [27]:
pw=''
driver = GraphDatabase.driver('bolt://robokopkg.renci.org:7687', auth=('neo4j', pw))

In [30]:
#cypher = f'MATCH (a:`biolink:Gene`) RETURN a LIMIT 1'
cypher = f"MATCH (n0_0:`biolink:ChemicalEntity`)-[r0_0]-(n1_0:`biolink:Gene`)-[r1_0]-(n2_0:`biolink:DiseaseOrPhenotypicFeature`), (n0_1:`biolink:ChemicalEntity`)-[r0_1]-(n1_1:`biolink:BiologicalProcessOrActivity`)-[r1_1]-(n2_1:`biolink:DiseaseOrPhenotypicFeature`), (n0_2:`biolink:ChemicalEntity`)-[r0_2]-(n1_2:`biolink:Pathway`)-[r1_2]-(n2_2:`biolink:DiseaseOrPhenotypicFeature`) WHERE n1_0.name IN ['CYP2D6'] AND n0_0.name IN ['Buprenorphine'] AND n2_0.name IN ['Tremor'] AND n0_1.name IN ['Buprenorphine'] AND n2_1.name IN ['Tremor'] AND n0_2.name IN ['Buprenorphine'] AND n2_2.name IN ['Tremor'] RETURN * LIMIT 100"

with driver.session() as session:
    results = session.run(cypher)
    for result in results:
        print(result)

    #print(results)
    if (results) == 0:
        print("No results found")



You can also send the cypher through the automat interface instead:

In [31]:
j = {'query': cypher}
results = requests.post('https://automat.renci.org/robokopkg/cypher',json=j)

In [32]:
print(results.json())

{'results': [{'columns': ['n0_0', 'n0_1', 'n0_2', 'n1_0', 'n1_1', 'n1_2', 'n2_0', 'n2_1', 'n2_2', 'r0_0', 'r0_1', 'r0_2', 'r1_0', 'r1_1', 'r1_2'], 'data': []}], 'errors': []}
